In [ ]:
import customtkinter as ctk
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk
from ultralytics import YOLO
import numpy as np

# --- GÖRÜNÜM AYARLARI ---
ctk.set_appearance_mode("Dark")
ctk.set_default_color_theme("blue")

class GuvenlikPaneli(ctk.CTk):
    def __init__(self):
        super().__init__()

        # Pencere Ayarları
        self.title("İnşaat Güvenlik Sistemi - Fotoğraf & Video")
        self.geometry("1280x720")
        
        # Değişkenler
        self.cap = None
        self.is_running = False
        
        # --- DÜZELTİLEN KISIM ---
        # Artık sadece dosya adını yazıyoruz (yanında olduğu sürece çalışır)
        self.model_path = "best.pt" 
        # ------------------------
        
        # Modeli Yükle
        self.model = self.model_yukle()

        # Arayüzü Çiz
        self.arayuz_tasarla()

    def model_yukle(self):
        print("Yapay Zeka Modeli Yükleniyor...")
        try:
            model = YOLO(self.model_path)
            tr_mapping = {
                'Hardhat': 'Baret', 'Mask': 'Maske', 'NO-Hardhat': 'Baret Yok',
                'NO-Mask': 'Maske Yok', 'NO-Safety Vest': 'Yelek Yok', 'Person': 'Insan',
                'Safety Cone': 'Duba', 'Safety Vest': 'Is Yelegi', 
                'machinery': 'Is Makinesi', 'vehicle': 'Arac', 'background': 'Arka Plan'
            }
            new_names = {id: tr_mapping.get(name, name) for id, name in model.names.items()}
            model.model.names = new_names
            print("Model Hazır!")
            return model
        except Exception as e:
            print(f"HATA: Model yüklenemedi! {e}")
            return None

    def arayuz_tasarla(self):
        # --- SOL MENÜ ---
        self.sidebar = ctk.CTkFrame(self, width=250, corner_radius=0)
        self.sidebar.pack(side="left", fill="y")

        self.lbl_logo = ctk.CTkLabel(self.sidebar, text="GÜVENLİK\nKONTROL", font=ctk.CTkFont(size=24, weight="bold"))
        self.lbl_logo.pack(pady=40)

        # Video Butonu
        self.btn_video = ctk.CTkButton(self.sidebar, text="Video Yükle", command=self.video_sec, height=40, font=ctk.CTkFont(size=15))
        self.btn_video.pack(padx=20, pady=10)

        # Fotoğraf Butonu
        self.btn_foto = ctk.CTkButton(self.sidebar, text="Fotoğraf Yükle", command=self.foto_sec, height=40, fg_color="#D35400", hover_color="#A04000", font=ctk.CTkFont(size=15))
        self.btn_foto.pack(padx=20, pady=10)

        # Durdur Butonu
        self.btn_dur = ctk.CTkButton(self.sidebar, text="Temizle / Durdur", command=self.durdur, height=40, fg_color="#C0392B", hover_color="#922B21", font=ctk.CTkFont(size=15))
        self.btn_dur.pack(padx=20, pady=30)

        # Durum Göstergesi
        self.lbl_durum_baslik = ctk.CTkLabel(self.sidebar, text="SİSTEM DURUMU:", font=ctk.CTkFont(size=14, weight="bold"))
        self.lbl_durum_baslik.pack(side="bottom", pady=(0, 5))
        
        self.lbl_durum = ctk.CTkLabel(self.sidebar, text="BEKLENİYOR...", text_color="gray", font=ctk.CTkFont(size=16))
        self.lbl_durum.pack(side="bottom", pady=(0, 30))

        # --- ORTA EKRAN ---
        self.video_frame = ctk.CTkFrame(self, corner_radius=10, border_width=5, border_color="#333")
        self.video_frame.pack(side="right", fill="both", expand=True, padx=20, pady=20)

        self.video_label = ctk.CTkLabel(self.video_frame, text="")
        self.video_label.pack(fill="both", expand=True, padx=5, pady=5)

    def foto_sec(self):
        self.durdur() # Önceki işlemi temizle
        dosya_yolu = filedialog.askopenfilename(filetypes=[("Resim Dosyaları", "*.jpg *.jpeg *.png *.bmp")])
        if dosya_yolu:
            self.foto_isle(dosya_yolu)

    def foto_isle(self, yol):
        # Resmi Oku
        frame = cv2.imread(yol)
        if frame is None:
            return

        # Yapay Zeka Analizi (Tek kare olduğu için 'predict' kullanıyoruz)
        results = self.model.predict(frame, conf=0.50, verbose=False)
        annotated_frame = results[0].plot()

        # İhlal Kontrolü
        ihlal_var = False
        yasakli_siniflar = ['Baret Yok', 'Yelek Yok']
        
        for box in results[0].boxes:
            cls_name = self.model.names[int(box.cls[0])]
            if cls_name in yasakli_siniflar:
                ihlal_var = True
                break

        # Arayüzü Güncelle
        self.gorsel_guncelle(annotated_frame, ihlal_var, is_video=False)

    def video_sec(self):
        dosya_yolu = filedialog.askopenfilename(filetypes=[("Video Dosyaları", "*.mp4 *.avi *.mkv")])
        if dosya_yolu:
            self.baslat_video(dosya_yolu)

    def baslat_video(self, kaynak):
        self.durdur()
        self.cap = cv2.VideoCapture(kaynak)
        self.is_running = True
        self.video_dongusu()

    def durdur(self):
        self.is_running = False
        if self.cap:
            self.cap.release()
        self.video_label.configure(image="")
        self.video_frame.configure(border_color="#333")
        self.lbl_durum.configure(text="DURDURULDU", text_color="gray")

    def video_dongusu(self):
        if not self.is_running:
            return

        ret, frame = self.cap.read()
        if not ret:
            self.durdur()
            return

        # Analiz
        results = self.model.track(frame, persist=True, conf=0.50, verbose=False)
        annotated_frame = results[0].plot()

        # Kontrol
        ihlal_var = False
        yasakli_siniflar = ['Baret Yok', 'Yelek Yok']
        for box in results[0].boxes:
            cls_name = self.model.names[int(box.cls[0])]
            if cls_name in yasakli_siniflar:
                ihlal_var = True
                break
        
        self.gorsel_guncelle(annotated_frame, ihlal_var, is_video=True)

    def gorsel_guncelle(self, frame_bgr, ihlal_var, is_video):
        # Renk ve Yazı Ayarı
        if ihlal_var:
            self.video_frame.configure(border_color="#E74C3C") # Kırmızı
            self.lbl_durum.configure(text="!!! GÜVENLİK İHLALİ !!!", text_color="#E74C3C")
        else:
            self.video_frame.configure(border_color="#2ECC71") # Yeşil
            self.lbl_durum.configure(text="GÜVENLİ", text_color="#2ECC71")

        # Görüntüyü Ekrana Basma
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(frame_rgb)

        panel_w = self.video_frame.winfo_width()
        panel_h = self.video_frame.winfo_height()
        
        if panel_w > 10 and panel_h > 10:
            img_pil.thumbnail((panel_w, panel_h), Image.LANCZOS)
        
        img_tk = ctk.CTkImage(light_image=img_pil, dark_image=img_pil, size=img_pil.size)
        self.video_label.configure(image=img_tk)
        self.video_label.image = img_tk

        # Eğer videoysa döngüyü devam ettir
        if is_video and self.is_running:
            self.after(10, self.video_dongusu)

if __name__ == "__main__":
    app = GuvenlikPaneli()
    app.mainloop()

Yapay Zeka Modeli Yükleniyor...
Model Hazır!


c:\Users\lenovo\anaconda3\envs\tid_env\lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'str'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
